# BUILD FEATURES

#### Load libraries

In [1]:
import os
import time
import numpy as np
import pandas as pd
from scipy.io import wavfile as wv
from scipy import signal
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft
from scipy.signal import welch
from scipy import stats
from detect_peaks import detect_peaks
from tqdm.notebook import trange, tqdm

In [2]:
import pywt

In [3]:
from collections import defaultdict, Counter

#### Load Dataset

In [4]:
X = np.load('data/processed/submission.npy')

#### Functions

In [5]:
sample_freq = 22050 

In [6]:
def preprocessing(raw_signal):

    # Rectification 
    rectified_signal = np.abs(raw_signal)

    # Low-pass filtering -> envelope
    sample_freq = 22050 # Hz
    cutoff_freq = 5     # Hz
    norm_cutoff_freq = cutoff_freq / (sample_freq / 2.0)
    b, a = signal.butter(2, norm_cutoff_freq, 'low')
    envelope = signal.filtfilt(b, a, rectified_signal, axis=0)

    return envelope, rectified_signal

In [7]:
def calculate_statistics(raw_signal):
    n5 = np.nanpercentile(raw_signal, 5)
    n25 = np.nanpercentile(raw_signal, 25)
    n75 = np.nanpercentile(raw_signal, 75)
    n95 = np.nanpercentile(raw_signal, 95)
    median = np.nanpercentile(raw_signal, 50)
    mean = np.nanmean(raw_signal)
    std = np.nanstd(raw_signal)
    var = np.nanvar(raw_signal)
    rms = np.nanmean(np.sqrt(raw_signal**2))
    maxv =np.max(raw_signal)
    minv = np.min(raw_signal)
    skew = stats.skew(raw_signal)
    kurtosis = stats.kurtosis(raw_signal)
    absmean = np.abs(raw_signal).mean()
    absstd = np.abs(raw_signal).std()
    
    
    return [n5, n25, n75, n95, median, mean, std, var, rms, maxv, minv, skew, kurtosis, absmean, absstd]

In [8]:
def calculate_signal_features(raw_signal):
    relamp = np.max(raw_signal) / np.abs(np.min(raw_signal))
    amp = np.max(raw_signal) - np.abs(np.min(raw_signal))
    ssum = np.sum(raw_signal)
    diffmean = np.mean(np.diff(raw_signal))
    
    return [relamp, amp, ssum, diffmean]

In [ ]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[len(result)//2:]

def get_autocorr_values(y_values, f_s):
    N = len(y_values)
    T = 1/f_s
    autocorr_values = autocorr(y_values)
    x_values = np.array([T * jj for jj in range(0, N)])
    return x_values, autocorr_values

In [ ]:
def get_fft_values(y_values, f_s):
    T = 1/f_s
    N = len(y_values)
    f_values = np.linspace(0.0, 1.0/(2.0*T), N//2)
    fft_values_ = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values_[0:N//2])
    return f_values, fft_values

In [ ]:
def get_psd_values(y_values, f_s):
    f_values, psd_values = welch(y_values, fs=f_s)
    return f_values, psd_values

In [ ]:
def get_first_n_peaks(x,y,no_peaks=5):
    x_, y_ = list(x), list(y)
    if len(x_) > no_peaks:
        return x_[:no_peaks], y_[:no_peaks]
    else:
        missing_no_peaks = no_peaks-len(x_)
        return x_ + [0]*missing_no_peaks, y_ + [0]*missing_no_peaks
    
def get_peaks(x_values, y_values, mph):
    indices_peaks = detect_peaks(y_values, mph=mph)
    peaks_x, peaks_y = get_first_n_peaks(x_values[indices_peaks], y_values[indices_peaks])
    return peaks_x + peaks_y

In [ ]:
def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

In [ ]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy = stats.entropy(probabilities)
    return entropy

In [ ]:
def get_features(raw_signal, f_s):
    
    envelope, rectified_signal = preprocessing(raw_signal)
    
    threshold = 0.0115 * (2 ** 16) # to compute duration and envelope slope sign
    above_thres = envelope >= threshold  # samples to consider in computation
    
    max_pos = np.argmax(envelope)
    envelope_slope_sign = np.sign(np.diff(envelope, append=0)) * above_thres    
    duration_signal = np.ones(envelope.shape) * above_thres

    # features
    features = []
    feature_names = []
    
    features += calculate_statistics(raw_signal)
    feature_names += ['rsig_' + i for i in ['n5', 'n25', 'n75', 'n95', 'median', 'mean', 'std', 'var', 'rms', 'maxv', 'minv', 'skew', 'kurtosis', 'absmean', 'absstd']]
    
    features += calculate_signal_features(raw_signal)
    feature_names += ['rsig_' + i for i in ['relamp', 'amp', 'ssum', 'diffmean']]
    
    features += calculate_crossings(raw_signal)
    feature_names += ['rsig_' + i for i in ['no_zero_crossings', 'no_mean_crossings']]
    
    features += calculate_statistics(get_fft_values(raw_signal, f_s)[1])
    feature_names += ['rsig_fft_' + i for i in ['n5', 'n25', 'n75', 'n95', 'median', 'mean', 'std', 'var', 'rms', 'maxv', 'minv', 'skew', 'kurtosis', 'absmean', 'absstd']]
    
    features += calculate_statistics(get_psd_values(raw_signal, f_s)[1])
    feature_names += ['rsig_psd_' + i for i in ['n5', 'n25', 'n75', 'n95', 'median', 'mean', 'std', 'var', 'rms', 'maxv', 'minv', 'skew', 'kurtosis', 'absmean', 'absstd']]

    
    # peaks
    percentile = 5
    denominator = 10
    
            
    signal_min = np.nanpercentile(raw_signal, percentile)
    signal_max = np.nanpercentile(raw_signal, 100-percentile)
    mph = signal_min + (signal_max - signal_min)/denominator

    peaks_detected = get_peaks(*get_psd_values(raw_signal, f_s), mph)
    features += peaks_detected
    feature_names += ['peak_psd_' + str(i) for i in range(len(peaks_detected))]
    
    peaks_detected = get_peaks(*get_fft_values(raw_signal, f_s), mph)
    features += peaks_detected
    feature_names += ['peak_fft_' + str(i) for i in range(len(peaks_detected))]
    
    
    peaks_detected = get_peaks(*get_autocorr_values(raw_signal, f_s), mph)
    features += peaks_detected
    feature_names += ['peak_corr_' + str(i) for i in range(len(peaks_detected))]
    
    # wavelet
    wave_families = ['haar', 'bior1.5', 'coif1', 'db1', 'sym2']
    for waveletname in wave_families:
        list_coeff = pywt.wavedec(raw_signal, waveletname)
        for n_coef, coeff in enumerate(list_coeff):
            features += [calculate_entropy(coeff)]
            feature_names += ['entro_' + waveletname + '_coeff_' + str(n_coef)]
            
            features += calculate_crossings(coeff)
            feature_names += [waveletname + '_' + i + '_coeff_' + str(n_coef) for i in ['no_zero_crossings', 'no_mean_crossings']]
            
            features += calculate_statistics(coeff)
            feature_names += [waveletname + '_' + i + '_coeff_' + str(n_coef) for i in ['n5', 'n25', 'n75', 'n95', 'median', 'mean', 'std', 'var', 'rms', 'maxv', 'minv', 'skew', 'kurtosis', 'absmean', 'absstd']]


    
    duration = np.sum(duration_signal)
    zero_crossing = np.count_nonzero(np.abs(np.diff(envelope_slope_sign)))
    amplitude = max(rectified_signal)
    ratio = np.trapz(envelope[:max_pos], axis=0) / np.trapz(envelope, axis=0)

    features += [duration, zero_crossing, float(amplitude), float(ratio)]
    feature_names += ['duration', 'zero_crossing', 'amplitude', 'ratio']
    
    
    
    
    return features, feature_names

#### Make features

In [ ]:
# Inicialize X zeros array 
X_features = [] #np.zeros([X.shape[0], 85])
feature_names = []
for idx in trange(X.shape[0]):
    #X_features[idx, :] = get_features(X[idx, :], sample_freq)
    if idx == 0:
        features, feature_names_ = get_features(X[idx, :], sample_freq)
        X_features.append(features)
        feature_names = feature_names_
    else:
        X_features.append(get_features(X[idx, :], sample_freq)[0])
        pass
    pass
X_features = np.array(X_features)

In [ ]:
X_features.shape

#### Save data

In [ ]:
np.save('data/features/submission_002.npy',X_features)

In [4]:
X_features = np.load('data/features/X_features_002.npy')

In [5]:
feature_names = np.load('data/features/X_features_names_002.npy')

In [7]:
[i for i, name in enumerate(feature_names) if name == 'duration']

[1269]